In [ ]:
# %% md
# # 02  Back-test Baselines
# Compare TWAP, AC, OW, Hawkes-LQ on toy order-flow
# %%
import numpy as np, pandas as pd, os, matplotlib.pyplot as plt
os.makedirs("../docs/figures", exist_ok=True)
os.makedirs("../docs/tables", exist_ok=True)
# %% 1. load order-flow
trace = pd.read_csv("../docs/figures/sample_orderflow.csv")
T, x0 = 3_600, 100_000
# %% 2. strategy wrappers
import sys; sys.path.append("..")
from backtest.backtester import (twap_strategy, ac_strategy, ow_strategy, hawkes_lq_strategy,
                                 compare_strategies)
# pre-compute Hawkes-LQ gains
from models.hawkes_control import compute_feedback_gains
gains = compute_feedback_gains(
        propagator_alphas=np.array([5e-4,2e-4]),
        propagator_betas=np.array([10.,50.]),
        propagator_rho=5.0,
        hawkes_alphas=np.array([0.2,0.15]),
        hawkes_betas=np.array([3.,3.]),
        hawkes_A=np.array([[0.2,0.15],[0.15,0.2]]),
        eta=1e-4, gamma=5e-5, theta=1e-4, T=T, N=120)
strategies = {
    'TWAP':  (twap_strategy,  dict(N=120, eta=1e-4, gamma=5e-5)),
    'AC':    (ac_strategy,    dict(eta=1e-4, gamma=5e-5, sigma=0.02, N=120)),
    'OW':    (ow_strategy,    dict(rho=1/30, eta=1e-4, pi=5e-5, N=120)),
    'Hawkes-LQ': (hawkes_lq_strategy, dict(
        gains=gains,
        propagator_alphas=np.array([5e-4,2e-4]),
        propagator_betas=np.array([10.,50.]),
        propagator_rho=5.0,
        hawkes_betas=np.array([3.,3.]),
        hawkes_A=np.array([[0.2,0.15],[0.15,0.2]]),
        eta=1e-4, gamma=5e-5))
# %% 3. run
summary = compare_strategies(trace, strategies, x0, T)
# %% 4. Table 2
summary.to_latex("../docs/tables/table2_strategy_comparison.tex", index=False, float_format="%.2f")
summary
# %% 5. Figures 3–5
from plots.plot_execution import plot_inventory_paths, plot_cost_distribution
plot_inventory_paths(); plot_cost_distribution()